In [ ]:
# Imports

import io
import pickle
import pandas as pd
import numpy as np
from PIL import Image

import tensorflow as tf
import tensorflow_hub as hub


In [ ]:
# Instantiating ResNet Model

model_url = "https://tfhub.dev/google/imagenet/resnet_v2_101/feature_vector/5"

IMAGE_SHAPE = (224, 224)

layer = hub.KerasLayer(model_url, input_shape = IMAGE_SHAPE + (3, ))
model = tf.keras.Sequential([layer])


In [ ]:
# Load PKL file containing downsized scraped images

with open("/content/drive/MyDrive/Recommender_project_2/image_results_downsized/images_results_41_downsized.pickle", "rb") as file:
    
    loaded_dict = pickle.load(file)


In [ ]:
'''
For each game in the pickle
For each image in the game images array
Open the byte array to image in RGB
Resize the image accordingly
Convert to numpy array
Use the model to get the embeddings (features)
Flatten the features
Append it to the final results
'''

results = []

for k,v in loaded_dict.items():
    
    for i, each_image in enumerate(v):
        
        image = Image.open(io.BytesIO(each_image)).convert('RGB')
        
        #image.show()
        
        file = image.resize(IMAGE_SHAPE)
        
        pix = np.array(file)
        
        embedding = model.predict(pix[np.newaxis, ...])
        
        embedding_np = np.array(embedding)
        
        flattended_feature = embedding_np.flatten()
        
        results.append({'game_id': str(k), 'image_sub_id': str(k) + '_' + str(i), 'vector': flattended_feature})

df = pd.DataFrame(results)

df.head()


In [ ]:
## Save the vectors to a PKL file

df.to_pickle('/content/drive/MyDrive/Recommender_project_2/image_results/vectors_41.pkl')
